<a href="https://colab.research.google.com/github/ZeevBB/Bitcoin-Trader-RL/blob/master/1_2_musig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setup Google Colab environment

This notebook should be run on [Google Colab](https://colab.research.google.com/) only. For local usage use the notebook provided [here](https://github.com/bitcoinops/taproot-workshop). 

Run the following cell to setup the Google Colab enviroment for this notebook.

In [1]:
!curl -o- -s -S -L https://raw.githubusercontent.com/bitcoinops/taproot-workshop/Colab/setup-colab-env.sh | bash

Colab environment setup.


In [2]:
import util
from test_framework.key import ECKey, ECPubKey, generate_key_pair, generate_schnorr_nonce
from test_framework.messages import sha256
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, jacobi_symbol, SECP256K1_FIELD_SIZE, sign_musig

Source directory configured as /content/bitcoin/


# 1.2 Introduction to n-of-n MuSig

* Public Key Generation
* Signing
    * Nonce Aggregation
    * Signature Aggregation

In this chapter, we introduce the interactive [MuSig protocol](https://eprint.iacr.org/2018/068.pdf) which allows n-of-n participants to jointly create and spend taproot or tapscript outputs using aggregated schnorr signatures. 

Using a signature aggregation scheme like MuSig has two significant advantages over using Script's `OP_CHECKMULTISIG` and tapscript's `OP_CHECKSIGADD` opcodes:

* **Transaction Size/Fees**: an aggregate MuSig pubkey and signature is indistinguishable from a single-key pubkey and signature, meaning that the transaction size (and required fee) for a multi-key output are the same as for a single-key output.
* **Privacy and Fungibility**: an aggregate MuSig pubkey and signature is indistinguishable from a single-key pubkey and signature, making it impossible for anyone to use the public block chain data to identify where a multi-key scheme has been used.

The MuSig protocol covers both the initial setup (generating an aggregate pubkey for all participants), and the signing protocol (creating a valid signature for the aggregate pubkey). The signing requires multiple rounds of communication between the individual signers.

Bip-schnorr is linear in the nonce points and public keys, which means that public keys, nonces and signatures can be aggregated. A very naive multiparty signature scheme could be achieved by simply summing the individual pubkeys to generate an aggregate pubkey, each participant signing with a shared nonce, and then summing the signatures. Such a scheme would be vulnerable to both the [key cancellation attack](https://tlu.tarilabs.com/cryptography/digital_signatures/introduction_schnorr_signatures.html#key-cancellation-attack) and private key extraction by exploiting weak or known nonces. Countering these attacks is what adds some complexity to the MuSig protocol.

![test](https://github.com/bitcoinops/taproot-workshop/blob/Colab/images/musig_intro_0.jpg?raw=1)

## Public Key Generation

To counter the key cancellation attack, each participant's pubkey is _tweaked_ by a _challenge factor,_ which is generated by hashing all the participants' pubkeys together. Doing this ensures that no individual participant (or group of participants) is able to create a pubkey that cancels out other participants' pubkeys.

The challenge factor is unique for each participant, but all challenge factors are based on a hash of all participants' pubkeys.

No interactive round-trips are required in this step. As long as everyone can get all the public keys involved (through communication, a coordinator or offline) then they can compute the challenge factors and the aggregate public key locally.

![test](https://github.com/bitcoinops/taproot-workshop/blob/Colab/images/musig_intro_1.jpg?raw=1)

#### 1.2.1 _Programming Exercise:_ Compute 3-of-3 MuSig public key

In this exercise, we'll use the `generate_musig_key(pubkey_list)` function to generate challenge factors for each participant and an aggregate MuSig pubkey.

`generate_musig_key(pubkey_list)` takes a list of the participants' public keys `generate_musig_key([ECPubKey0, ECPubKey1, ...])` and returns a challenge map and the aggregate pubkey:
* The challenge map contains `ECPubKey_i, challenge_data_i` key - value pairs.
* The aggregate pubkey is an `ECPubKey` object.

In [5]:
# Compute key pairs
privkey1, pubkey1 = generate_key_pair(sha256(b'key0'))
privkey2, pubkey2 = generate_key_pair(sha256(b'key1'))
privkey3, pubkey3 = generate_key_pair(sha256(b'key2'))
pubkeys = [pubkey1, pubkey2, pubkey3]

# Compute key challenges
# Method: use generate_musig_key() on the list of pubkeys.
# generate_musig_key() returns a challenge map and the aggregate public key.
#c_map, pubkey_agg =  # TODO: implement
c_map, pubkey_agg = generate_musig_key(pubkeys)
print("Aggregated Public Key is {}\n".format(pubkey_agg.get_bytes().hex()))
print("cPmap tyye", type(c_map), "c_map", c_map)
# Multiply key pairs by challenge factor
#privkey1_c =  # TODO: implement
privkey1_c = privkey1 * c_map[pubkey1]
#privkey2_c =  # TODO: implement
privkey2_c = privkey2 * c_map[pubkey2]
#privkey3_c =  # TODO: implement
privkey3_c = privkey3 * c_map[pubkey3]
#pubkey1_c =  # TODO: implement
pubkey1_c =  pubkey1 * c_map[pubkey1]
#pubkey2_c =  # TODO: implement
pubkey2_c =  pubkey2 * c_map[pubkey2]
#pubkey3_c =  # TODO: implement
pubkey3_c =  pubkey3 * c_map[pubkey3]

print("Tweaked privkey1 is {}".format(privkey1_c))
print("Tweaked privkey2 is {}".format(privkey2_c))
print("Tweaked privkey3 is {}".format(privkey3_c))

assert privkey1_c.secret == 104717570570407299858230629579807834166658508605015363884161538594382975780625
assert privkey2_c.secret == 65554880484297966965546994775376394861215085064604177497808278620612854069980
assert privkey3_c.secret == 106998690642216524894360365246223287721822845133760006050846956016514597569168

print("\nSuccess!")

Aggregated Public Key is 02eeeea7d79f3ecde08d2a3c59f40eb3adcac9defb77d3b92053e5df95165139cd

cPmap tyye <class 'dict'> c_map {026c5d5e73124f3c821c0985df787e11b3d018a86add577fa8661613a0d49dde59: b'\xf3\xae.\xb9\xcb\xc0\xc3\xf8\xde\xfb\xb0\tZ\x17BY-QiC]\x0e\xf3\xbdJVZ\xea\x94s\xfd%', 03f771877964fa2ce401d87bc2558a0df1e6921acef99389f059712b32cfda35fd: b'P\xae\xcd\xba\x16\xff\xe44*X\xcf\xd6i\x10\x04D\x18\xb6*\xdc\x00\x9b\xb1N\x94\x07\xca6 i\x82\x98', 03f039fdcdb728efbbddf4ee452419a988497debb7bd1b42644c5fa66e9af8c8b6: b'\xbf\xc2W<t\x0b(5 \x17~\xe6\x1bK\xa7\x1a\xb6&\xceop\x05C\x04\x1f$\xecM\xa1g\x97\x89'}
Tweaked privkey1 is 104717570570407299858230629579807834166658508605015363884161538594382975780625
Tweaked privkey2 is 65554880484297966965546994775376394861215085064604177497808278620612854069980
Tweaked privkey3 is 106998690642216524894360365246223287721822845133760006050846956016514597569168

Success!


## Signing 

### Nonce generation

The first step of creating a MuSig signature requires each signer to generate their own nonce and nonce point. The participants then exchange those nonces points and an aggregate nonce point is derived by summing all the nonce points.

The security proof for MuSig requires that nonces are randomly generated and are independent of each other. To ensure that no individual participant (or group of participants) can create their nonce as a function of the other nonces or individually control what the aggregate nonce point will be, there is an initial round of exchanging hash commitments to the individual nonce points.

Individual participants should only exchange their nonce point when they have received all commitments, and only proceed with signing if all nonce points match their commitments.

Finally, if the aggregate nonce is not a quadratic residue, then it is negated and all individual nonces are also negated.

![test](https://github.com/bitcoinops/taproot-workshop/blob/Colab/images/musig_intro_2.jpg?raw=1)

#### 1.2.2 _Programming Exercise:_ Compute 3-of-3 MuSig nonce

In this exercise, we'll generate nonces for individual participants, calculate the nonce point commitments, and then generate an aggregate nonce point.

In [7]:
# Generate nonces and nonce points
# We set the nonces manually here for testing purposes, but usually we'll call generate_schnorr_nonce()
# to generate a random nonce point
# Method: generate_schnorr_nonce() with no argument generates a random nonce
k1 = ECKey().set(101)
k2 = ECKey().set(222)
k3 = ECKey().set(333)
test_k1 = ECKey().set(k1.secret)
test_k2 = ECKey().set(k2.secret)
test_k3 = ECKey().set(k3.secret)

# Method: use get_pubkey() to get the associated nonce point.
#R1 =  # TODO: implement
R1 = k1.get_pubkey()
#R2 =  # TODO: implement
R2 = k2.get_pubkey()
#R3 =  # TODO: implement
R3 = k3.get_pubkey()

# Round 1: Generate nonce point commitments and exchange them
# Method: use sha256() on the nonce point. sha256() takes a bytes object, so extract the bytes from the nonce point.
#R1_digest =  # TODO: implement
R1_digest = sha256(R1.get_bytes())
#R2_digest =  # TODO: implement
R2_digest = sha256(R2.get_bytes())
#R3_digest =  # TODO: implement
R3_digest = sha256(R3.get_bytes())

# Round 2: Exchange the nonce points. Each participant verifies that the nonce point commitment matches the nonce point.
assert R1_digest.hex() == "aa5d4a40c1843456534d75f7246b9cbf0f825a36113102e76193f80e6c652c72"
assert R2_digest.hex() == "6a78f2c6ca3bf1364f0575a02ef334d176df2051084c0e3062ec2af79b5b406a"
assert R3_digest.hex() == "bc2c379a7b6ad82b40a7cf280697505ccc039370e50eb155324dcac3b5faa6a5"

# Aggregate nonces
# Tip: Add the individual nonce points together. If the aggregated nonce is not a quadratic residue
# then negate the aggregate nonce and individual nonce scalars.
#R_agg =  # TODO: implement
#R_agg = R1_digest + R2_digest + R3_digest 
R_agg, negated = aggregate_schnorr_nonces([R1, R2, R3])
#if  # TODO: implement
if negated:
    k1.negate()
    k2.negate()
    k3.negate()

print("Individual nonce scalars:\n\t{}, \n\t{}, \n\t{}.\n".format(k1, k2, k3))
print("Aggregate nonce point: {}\n".format(R_agg))

# Test your solution against the aggregate_schnorr_nonces() helper function.
# aggregate_schnorr_nonces() aggregates the nonces and returns whether the individual nonces need to be negated.
test_R_agg, negated = aggregate_schnorr_nonces([test_k1.get_pubkey(), test_k2.get_pubkey(), test_k3.get_pubkey()])
if negated:
    test_k1.negate()
    test_k2.negate()
    test_k3.negate()

assert R_agg == test_R_agg
assert k1 == test_k1
assert k2 == test_k2
assert k3 == test_k3

print("Success!")

Individual nonce scalars:
	115792089237316195423570985008687907852837564279074904382605163141518161494236, 
	115792089237316195423570985008687907852837564279074904382605163141518161494115, 
	115792089237316195423570985008687907852837564279074904382605163141518161494004.

Aggregate nonce point: 03f90c3416d74049bf27b5563067c58401ff466e4bb04e1fa4d51ae4c93b4a8316

Success!


### Signature Aggregation

Once all participants have their individual nonces and the aggregate nonce point, then can all sign individually. 

The partial `s` values are then exchanged and summed together. The aggregate `s` value and aggregate nonce point `R` form a valid bip-schnorr signature for the aggregate pubkey.

Notice that the hash expressions are identical in all signatures, which makes aggregation possible.

In [4]:
help(jacobi_symbol)

Help on function jacobi_symbol in module test_framework.key:

jacobi_symbol(n, k)
    Compute the Jacobi symbol of n modulo k
    
    See http://en.wikipedia.org/wiki/Jacobi_symbol
    
    For our application k is always prime, so this is the same as the Legendre symbol.



![test](https://github.com/bitcoinops/taproot-workshop/blob/Colab/images/musig_intro_3.jpg?raw=1)

#### 1.2.3 _Programming exercise:_ Compute aggregated MuSig signature

In this exercise, we'll create partial signatures and then aggregate them to create a valid signature.

Use the `sign_musig()` function to create partial signatures. `sign_musig()` takes:
  - the individual participant's challenge-factor-modified private key (an `ECKey` object)
  - the invididual participant's nonce scalar (an `ECKey` object)
  - the aggregate nonce point (an `ECPubKey` object)
  - the aggregate pubkey (an `ECPubKey` object)
  - the message (a 32 byte `bytes` object)

and returns a partial signature (an int containing the partial `s` value).

Use `aggregate_musig_signatures()` to aggregate the partial signatures. `aggregate_musig_signatures()` takes a list of partial signatures, and the aggregate nonce point and returns the final signature.

Use `ECPubKey.verify_schnorr(sig, msg)` to verify that the signature is valid.

In [9]:
msg = sha256(b'transaction')

# Generate partial signatures
# Method: use sign_musig() with:
#     - individual (tweaked) privkey
#     - individual nonce scalar
#     - aggregate nonce point
#     - aggregate pubkey
#     - msg
#s1 =  # TODO: implement
s1 = sign_musig(privkey1_c, k1, R_agg, pubkey_agg, msg)
#s2 =  # TODO: implement
s2 = sign_musig(privkey2_c, k2, R_agg, pubkey_agg, msg)
#s3 =  # TODO: implement
s3 = sign_musig(privkey3_c, k3, R_agg, pubkey_agg, msg)

print("Partial signatures:\n\t{}\n\t{}\n\t{}\n".format(s1, s2, s3))

# Aggregate signatures
# Method: use aggregate_musig_signatures with list of individual signatures
#sig_agg =  # TODO: implement
sig_agg = aggregate_musig_signatures([s1,s2,s3], R_musig=R_agg)
print("Aggregate signature:\n\t{}\n".format(sig_agg.hex()))

# Verify signature against aggregate public key
assert pubkey_agg.verify_schnorr(sig_agg, msg)
print("Success! Signature verifies against aggregate pubkey")

Partial signatures:
	65632340538892058604021005685526525791383877758802541334726676556343496273695
	49071424722101348040708779394444974474178306070453713897027096230295229616215
	40052638870461774859002446004708555184684360378078494722527781424448839071327

Aggregate signature:
	f90c3416d74049bf27b5563067c58401ff466e4bb04e1fa4d51ae4c93b4a83165625054ca06a0e7a76ecca379955370d56fa014fc1c0e62313dd4ed246b23494

Success! Signature verifies against aggregate pubkey


**Congratulations!** In this chapter, you have:

- Learned about the MuSig multi-signature scheme.
- Computed an aggregate public key that multiplies individual public keys by _challenge factors_ to counter the rogue key attack.
- Exchanged nonce point commitments, and then the nonce points themselves, and generated an aggregate nonce point.
- Signed individually and then aggregated individual `s` values to create a valid signature for the aggregate public key.